# 1. EDA и выбор метрики

In [ ]:
# Загрузка данных и первичный осмотр
import pandas as pd

df = pd.read_csv('data_ab.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2025-01-21 22:11:48.556739,control,old_page,0.0
1,804228,2025-01-12 08:01:45.159739,control,old_page,0.0
2,661590,2025-01-11 16:55:06.154213,treatment,new_page,0.0
3,853541,2025-01-08 18:28:03.143765,treatment,new_page,0.0
4,864975,2025-01-21 01:52:26.210827,control,old_page,1.0


In [ ]:
# Проверка структуры датасета
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310688 entries, 0 to 310687
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   user_id       310688 non-null  int64  
 1   timestamp     310688 non-null  object 
 2   group         310687 non-null  object 
 3   landing_page  310687 non-null  object 
 4   converted     310687 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 11.9+ MB


,user_id,timestamp,group,landing_page,converted
count,310688.000000,310688,310687,310687,310687.000000
unique,NaN,291270,2,3,NaN
top,NaN,2025-01-20 10:40:36.863851,treatment,old_page,NaN
freq,NaN,2,155367,155383,NaN
mean,788000.500454,NaN,NaN,NaN,0.119876
std,91217.710351,NaN,NaN,NaN,0.324817
min,630000.000000,NaN,NaN,NaN,0.000000
25%,709043.750000,NaN,NaN,NaN,0.000000
50%,787968.500000,NaN,NaN,NaN,0.000000
75%,866935.250000,NaN,NaN,NaN,0.000000


In [ ]:
# Проверка корректности рандомизации
df['group'].value_counts(normalize=True)
df['converted'].mean()
df.groupby('group')['converted'].mean()
df.groupby('landing_page')['converted'].mean()

,converted
landing_page,
new_page,0.118832
old_page,0.120921
old_pageold_page,0.000000


Датасет содержит 77 670 наблюдений, в нём присутствуют две основные группы: old_page и new_page. Распределение по группам практически равное (≈50/50).

Общая конверсия составляет около 12%. Конверсия у old_page немного выше (≈0.12098), чем у new_page (≈0.11997), однако разница очень мала и не позволяет сделать выводы без статистической проверки.

В качестве ключевой метрики A/B-теста выбрана Conversion Rate (CR), отражающая долю пользователей, совершивших целевое действие.

# 2. Проверка корректности тестирования

In [ ]:
# Размер групп
df['group'].value_counts(normalize=True)

,proportion
group,
treatment,0.500803
control,0.499197


In [ ]:
df = df[
    ((df['group'] == 'control') & (df['landing_page'] == 'old_page')) |
    ((df['group'] == 'treatment') & (df['landing_page'] == 'new_page'))
]

# Проверка корректного сопоставления group и landing_page
df.groupby(['group', 'landing_page']).size()

,,0
group,landing_page,
control,old_page,47809
treatment,new_page,47956


In [ ]:
df = df[df['group'].isin(['control', 'treatment'])]

In [ ]:
# искусственно разбиваем группу пополам
import numpy as np

control = df[df['group'] == 'control'].copy()
control['aa_split'] = np.random.randint(0, 2, size=len(control))

control.groupby('aa_split')['converted'].mean()

,converted
aa_split,
0,0.118373
1,0.122769


In [ ]:
from statsmodels.stats.proportion import proportions_ztest

counts = [
    control[control['aa_split'] == 0]['converted'].sum(),
    control[control['aa_split'] == 1]['converted'].sum()
]

nobs = [
    len(control[control['aa_split'] == 0]),
    len(control[control['aa_split'] == 1])
]

stat, p = proportions_ztest(counts, nobs, alternative='two-sided')
stat, p

(np.float64(-1.476166173964529), np.float64(0.1398992877994047))

A/A-тест (проверка корректности рандомизации)

Для проверки однородности распределения пользователей был проведён A/A-тест: контрольная группа была случайным образом разделена на две подгруппы одинакового размера. Для сравнения конверсий между ними использовался z-тест для долей.

Результаты теста:

z-статистика = −1.476

p-value = 0.1399

Так как p-value значительно превышает уровень значимости 0.05, статистически значимых различий между подгруппами не обнаружено. Это говорит о том, что рандомизация была выполнена корректно, и группы пользователей однородны. Следовательно, данные пригодны для дальнейшего проведения A/B-теста.